# AAE 875 - Fundamentals of OOP and Data Analytics
# Week 5 - Summer 2019

<br>
<br>
<br>

My goal today is to give you a tour of the key Python packages used for **data visualization, descriptive statistics and regression analysis**. I am going to assume that you have already read the corresponding chapters in zyBooks.

The model we are going to run today will explore the cross-sectional infidelity data collected by the magazine Psychology Today in 1969. The economist Ray C. Fair used it the to develop "A Theory of Extramarital Affairs" (Fair, 1978).

The data consists of 601 observations on 9 variables as follows:

    Number of observations: 601
    Number of variables: 9
    Variable name definitions:


        sex             : Respondent gender: male or female
        age             : Respondent age
        yrs_married     : Number of years married
        children        : yes or no
        religious       : How relgious, 1 = not, 2 = mildly, 3 = fairly,
                        4 = strongly
        educ            : Level of education, 9 = grade school, 12 = high
                        school, 14 = some college, 16 = college graduate,
                        17 = some graduate school, 20 = advanced degree
        occupation      : 1 = student, 2 = farming, agriculture; semi-skilled,
                        or unskilled worker; 3 = white-colloar; 4 = teacher
                        counselor social worker, nurse; artist, writers;
                        technician, skilled worker, 5 = managerial,
                        administrative, business, 6 = professional with
                        advanced degree
       rate_marriage    : How rate marriage, 1 = very poor, 2 = poor, 3 = fair,
                        4 = good, 5 = very good
       nb_of_affairs    : number of extramarital affairs

    
    See the original paper for more details. 


We will use this data to answer the following questions:

**Data Visualization and Descriptive Statistics**

[1] Of all the people sampled, how many had affairs? How many women had affairs? How many men had affairs?

[2] Plot the age distribution of people who had identified as having at least one affair by sex.

[3] Plot the yrs_married distribution of people who had identified as having at least one affair by sex.

[4] Of all the people that had affairs, how many fall into each education category?

[5] Of all the people that had affairs, how many had children?

[5] For each religious rating, what was the average number of affairs?

[6] For each marriage satisfation rating, what was the average number of affairs?

[7] For each occupation category, what was the average number of affairs?

**Regression Analysis**

[1] Predict the probability of having an affair using the tools available in Python for discrete choice models (hint: statsmodels.formula.api). First fit a Logit model. Then fit a Tobit model. Can you conclude that the "correct" model is likely the Tobit model? Explain.

### Import packages

### Load affairs.csv data

### Print the number of rows and the number of columns

### Print the column labels

### Print the data type of each column

### Print the first 10 rows 

### Rename the x variable to resp_id

### Are there any missing values in the data?

### Print rows with missing values

### Remove rows with missing values

### What is the number of rows and columns now?

### Of all the people sampled, how many had affairs? How many women had affairs? How many men had affairs?

### Plot the age distribution of people who had identified as having at least one affair by sex.

### Plot the yrs_married distribution of people who had identified as having at least one affair by sex.

### Of all the people that had affairs, how many fall into each education category?

### Of all the people that had affairs, how many had children?

### For each religious rating, what was the average number of affairs?

### For each marriage satisfation rating, what was the average number of affairs?

### For each occupation category, what was the average number of affairs?

### Predict the probability of having an affair using the tools available in Python for discrete choice models

#### Fit a Logit model

#### Fit a Tobit model

#### Can you conclude that the "correct" model is likely the Tobit model? Explain.